In [1]:
import sqlite3
import pandas as pd
from flask import Flask, render_template, request, jsonify, url_for
import json
import urllib.parse
from datetime import datetime
import os
import threading
from IPython.display import display, HTML

In [2]:
# SQLite 데이터베이스 연결 및 테이블 생성
def create_database():
    conn = sqlite3.connect('korean_dict.db')
    cursor = conn.cursor()
    
    # 단어 테이블
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS words (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        word TEXT UNIQUE NOT NULL,
        pronunciation TEXT,
        origin TEXT,
        created_at DATETIME DEFAULT CURRENT_TIMESTAMP
    )
    ''')
    
    # 품사 테이블
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS parts_of_speech (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT UNIQUE NOT NULL,
        abbreviation TEXT
    )
    ''')
    
    # 정의 테이블
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS definitions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        word_id INTEGER,
        part_of_speech_id INTEGER,
        meaning TEXT NOT NULL,
        example TEXT,
        order_num INTEGER DEFAULT 1,
        FOREIGN KEY (word_id) REFERENCES words (id),
        FOREIGN KEY (part_of_speech_id) REFERENCES parts_of_speech (id)
    )
    ''')
    
    # 인덱스 생성 (검색 성능 향상)
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_word ON words(word)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_meaning ON definitions(meaning)')
    
    conn.commit()
    conn.close()
    print("데이터베이스가 성공적으로 생성되었습니다!")

create_database()

데이터베이스가 성공적으로 생성되었습니다!


In [3]:
def insert_parts_of_speech():
    conn = sqlite3.connect('korean_dict.db')
    cursor = conn.cursor()
    
    pos_data = [
        ('명사', '명'),
        ('동사', '동'),
        ('형용사', '형'),
        ('부사', '부'),
        ('관형사', '관'),
        ('감탄사', '감'),
        ('조사', '조'),
        ('어미', '어'),
    ]
    
    cursor.executemany('''
    INSERT OR IGNORE INTO parts_of_speech (name, abbreviation) 
    VALUES (?, ?)
    ''', pos_data)
    
    conn.commit()
    conn.close()
    print("품사 데이터가 입력되었습니다!")

insert_parts_of_speech()

품사 데이터가 입력되었습니다!


In [4]:
def insert_sample_data():
    conn = sqlite3.connect('korean_dict.db')
    cursor = conn.cursor()
    
    # 단어 삽입
    sample_words = [
        ('가을', '[가을]', '고유어'),
        ('봄', '[봄]', '고유어'),
        ('여름', '[여름]', '고유어'),
        ('겨울', '[겨울]', '고유어'),
        ('사랑', '[사랑]', '고유어'),
        ('희망', '[히망]', '한자어'),
        ('학교', '[학꾜]', '한자어'),
        ('컴퓨터', '[컴퓨터]', '외래어'),
    ]
    
    for word_data in sample_words:
        cursor.execute('''
        INSERT OR IGNORE INTO words (word, pronunciation, origin) 
        VALUES (?, ?, ?)
        ''', word_data)
    
    # 정의 삽입 (가을 예시)
    cursor.execute('SELECT id FROM words WHERE word = "가을"')
    word_id = cursor.fetchone()[0]
    
    cursor.execute('SELECT id FROM parts_of_speech WHERE name = "명사"')
    pos_id = cursor.fetchone()[0]
    
    definitions = [
        (word_id, pos_id, '여름과 겨울 사이의 계절. 음력으로는 7, 8, 9월이고, 양력으로는 9, 10, 11월이다.', 
         '가을이 되니 날씨가 선선해졌다.', 1),
        (word_id, pos_id, '사물이 무르익은 때를 비유적으로 이르는 말.', 
         '인생의 가을을 맞이하다.', 2),
    ]
    
    cursor.executemany('''
    INSERT OR IGNORE INTO definitions 
    (word_id, part_of_speech_id, meaning, example, order_num) 
    VALUES (?, ?, ?, ?, ?)
    ''', definitions)
    
    conn.commit()
    conn.close()
    print("샘플 데이터가 입력되었습니다!")

insert_sample_data()

샘플 데이터가 입력되었습니다!


In [5]:
app = Flask(__name__)

# 데이터베이스 연결 함수
def get_db_connection():
    conn = sqlite3.connect('korean_dict.db')
    conn.row_factory = sqlite3.Row
    return conn

# 메인 페이지
@app.route('/')
def index():
    return '''
    <!DOCTYPE html>
    <html>
    <head>
        <title>한국어 사전</title>
        <meta charset="UTF-8">
        <style>
            body { font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; }
            .search-box { text-align: center; margin: 50px 0; }
            .search-input { padding: 15px; font-size: 18px; width: 400px; border: 2px solid #ddd; }
            .search-btn { padding: 15px 30px; font-size: 18px; background: #4CAF50; color: white; border: none; cursor: pointer; }
            .logo { font-size: 48px; color: #4CAF50; margin-bottom: 30px; }
        </style>
    </head>
    <body>
        <div class="search-box">
            <div class="logo">한국어 사전</div>
            <form action="/search/m/" method="get">
                <input type="text" name="q" class="search-input" placeholder="검색할 단어를 입력하세요" required>
                <button type="submit" class="search-btn">검색</button>
            </form>
        </div>
    </body>
    </html>
    '''

# 단어 추가 API (여기에 미리 포함)
@app.route('/api/add_word', methods=['POST'])
def add_word():
    data = request.json
    word = data.get('word')
    pronunciation = data.get('pronunciation', '')
    origin = data.get('origin', '')
    meaning = data.get('meaning')
    pos_name = data.get('part_of_speech', '명사')
    
    conn = get_db_connection()
    
    try:
        # 단어 삽입
        cursor = conn.execute('''
        INSERT OR IGNORE INTO words (word, pronunciation, origin) 
        VALUES (?, ?, ?)
        ''', (word, pronunciation, origin))
        
        # 단어 ID 가져오기
        word_id = conn.execute('SELECT id FROM words WHERE word = ?', (word,)).fetchone()['id']
        
        # 품사 ID 가져오기
        pos_id = conn.execute('SELECT id FROM parts_of_speech WHERE name = ?', (pos_name,)).fetchone()['id']
        
        # 정의 삽입
        conn.execute('''
        INSERT INTO definitions (word_id, part_of_speech_id, meaning) 
        VALUES (?, ?, ?)
        ''', (word_id, pos_id, meaning))
        
        conn.commit()
        return jsonify({'status': 'success', 'message': '단어가 추가되었습니다.'})
    
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)})
    
    finally:
        conn.close()

# 검색 결과 페이지 (wordrow.kr 스타일)
@app.route('/search/m/')
def search_meaning():
    query = request.args.get('q', '').strip()
    if not query:
        return "검색어를 입력해주세요."
    
    conn = get_db_connection()
    
    # 단어와 의미에서 검색
    results = conn.execute('''
    SELECT DISTINCT w.word, w.pronunciation, w.origin,
                   d.meaning, d.example, p.name as pos_name, p.abbreviation
    FROM words w
    LEFT JOIN definitions d ON w.id = d.word_id
    LEFT JOIN parts_of_speech p ON d.part_of_speech_id = p.id
    WHERE w.word LIKE ? OR d.meaning LIKE ?
    ORDER BY w.word, d.order_num
    ''', (f'%{query}%', f'%{query}%')).fetchall()
    
    conn.close()
    
    if not results:
        return f'<h2>"{query}"에 대한 검색 결과가 없습니다.</h2>'
    
    # 결과를 단어별로 그룹화
    word_groups = {}
    for row in results:
        word = row['word']
        if word not in word_groups:
            word_groups[word] = {
                'word': word,
                'pronunciation': row['pronunciation'],
                'origin': row['origin'],
                'definitions': []
            }
        if row['meaning']:
            word_groups[word]['definitions'].append({
                'pos': row['pos_name'],
                'pos_abbr': row['abbreviation'],
                'meaning': row['meaning'],
                'example': row['example']
            })
    
    # HTML 생성
    html = f'''
    <!DOCTYPE html>
    <html>
    <head>
        <title>"{query}" 검색결과 - 한국어 사전</title>
        <meta charset="UTF-8">
        <style>
            body {{ font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; }}
            .header {{ text-align: center; margin-bottom: 30px; }}
            .search-input {{ padding: 10px; font-size: 16px; width: 300px; }}
            .word-entry {{ border: 1px solid #ddd; margin: 20px 0; padding: 20px; }}
            .word-title {{ font-size: 24px; font-weight: bold; color: #333; }}
            .pronunciation {{ color: #666; margin-left: 10px; }}
            .origin {{ color: #999; font-size: 14px; }}
            .definition {{ margin: 10px 0; padding: 10px; background: #f9f9f9; }}
            .pos {{ color: #4CAF50; font-weight: bold; }}
            .meaning {{ margin: 5px 0; }}
            .example {{ color: #666; font-style: italic; margin-top: 5px; }}
        </style>
    </head>
    <body>
        <div class="header">
            <h1><a href="/" style="text-decoration:none; color:#4CAF50;">한국어 사전</a></h1>
            <form action="/search/m/" method="get">
                <input type="text" name="q" class="search-input" value="{query}" required>
                <button type="submit">검색</button>
            </form>
        </div>
        
        <h2>"{query}" 검색결과 ({len(word_groups)}개)</h2>
    '''
    
    for word_data in word_groups.values():
        html += f'''
        <div class="word-entry">
            <div class="word-title">
                {word_data['word']}
                <span class="pronunciation">{word_data['pronunciation'] or ''}</span>
            </div>
            <div class="origin">{word_data['origin'] or ''}</div>
        '''
        
        for i, definition in enumerate(word_data['definitions'], 1):
            html += f'''
            <div class="definition">
                <span class="pos">{definition['pos_abbr'] or definition['pos']}</span>
                <div class="meaning">{i}. {definition['meaning']}</div>
                {f'<div class="example">예: {definition["example"]}</div>' if definition['example'] else ''}
            </div>
            '''
        
        html += '</div>'
    
    html += '''
    </body>
    </html>
    '''
    
    return html

print("Flask 앱이 설정되었습니다!")

Flask 앱이 설정되었습니다!


In [6]:
# Flask 앱 재시작 (기존 앱이 있다면 새로 생성)
try:
    app.shutdown()
except:
    pass

# 새로운 앱 인스턴스 생성
app = Flask(__name__)

# 모든 라우트를 다시 정의해야 합니다
# (위의 6번째 셀 내용을 다시 실행하거나, 커널을 재시작하고 처음부터 실행하세요)

# 또는 커널 재시작을 권장합니다
print("오류 해결을 위해 다음 중 하나를 선택하세요:")
print("1. 커널 재시작 후 처음부터 다시 실행 (권장)")
print("2. 아래 코드로 앱을 완전히 새로 생성")

오류 해결을 위해 다음 중 하나를 선택하세요:
1. 커널 재시작 후 처음부터 다시 실행 (권장)
2. 아래 코드로 앱을 완전히 새로 생성


In [7]:
# 기존 변수들 정리
import importlib
import sys
import requests
import xml.etree.ElementTree as ET
import urllib.parse

# Flask 앱을 완전히 새로 생성
app = Flask(__name__)

# 우리말샘 API 설정
OPENDICT_API_KEY = "561433A49188D3CB67FEDC2120EC2C87"  # 실제 API 키로 변경하세요
OPENDICT_API_URL = "https://opendict.korean.go.kr/api/search"

# 우리말샘 API 호출 함수
def search_opendict(query, num=10):
    """우리말샘 API를 호출하여 검색 결과를 가져옵니다."""
    params = {
        'key': OPENDICT_API_KEY,
        'req_type': 'json',
        'q': query,
        'num': num,
        'sort': 'dict',
        'part': 'word'
    }
    
    try:
        response = requests.get(OPENDICT_API_URL, params=params)
        response.raise_for_status()
        data = response.json()
        
        if 'channel' in data and 'item' in data['channel']:
            return data['channel']['item'] if isinstance(data['channel']['item'], list) else [data['channel']['item']]
        return []
    except Exception as e:
        print(f"API 호출 오류: {e}")
        return []

# 로컬 데이터베이스 연결 함수 (백업용)
def get_db_connection():
    conn = sqlite3.connect('korean_dict.db')
    conn.row_factory = sqlite3.Row
    return conn

# 메인 페이지
@app.route('/')
def index():
    return '''
    <!DOCTYPE html>
    <html>
    <head>
        <title>한국어 사전</title>
        <meta charset="UTF-8">
        <style>
            body { font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; }
            .search-box { text-align: center; margin: 50px 0; }
            .search-input { padding: 15px; font-size: 18px; width: 400px; border: 2px solid #ddd; }
            .search-btn { padding: 15px 30px; font-size: 18px; background: #4CAF50; color: white; border: none; cursor: pointer; }
            .logo { font-size: 48px; color: #4CAF50; margin-bottom: 30px; }
        </style>
    </head>
    <body>
        <div class="search-box">
            <div class="logo">한국어 사전</div>
            <form action="/search/m/" method="get">
                <input type="text" name="q" class="search-input" placeholder="검색할 단어를 입력하세요" required>
                <button type="submit" class="search-btn">검색</button>
            </form>
        </div>
    </body>
    </html>
    '''

# 검색 결과 페이지 (우리말샘 API 사용)
@app.route('/search/m/')
def search_meaning():
    query = request.args.get('q', '').strip()
    if not query:
        return "검색어를 입력해주세요."
    
    # 우리말샘 API에서 검색
    api_results = search_opendict(query)
    
    # API 결과가 없으면 로컬 데이터베이스에서 검색 (백업)
    if not api_results:
        conn = get_db_connection()
        local_results = conn.execute('''
        SELECT DISTINCT w.word, w.pronunciation, w.origin,
                       d.meaning, d.example, p.name as pos_name, p.abbreviation
        FROM words w
        LEFT JOIN definitions d ON w.id = d.word_id
        LEFT JOIN parts_of_speech p ON d.part_of_speech_id = p.id
        WHERE w.word LIKE ? OR d.meaning LIKE ?
        ORDER BY w.word, d.order_num
        ''', (f'%{query}%', f'%{query}%')).fetchall()
        conn.close()
        
        if not local_results:
            return f'<h2>"{query}"에 대한 검색 결과가 없습니다.</h2>'
        
        # 로컬 결과 처리 (기존 방식)
        word_groups = {}
        for row in local_results:
            word = row['word']
            if word not in word_groups:
                word_groups[word] = {
                    'word': word,
                    'pronunciation': row['pronunciation'],
                    'origin': row['origin'],
                    'definitions': []
                }
            if row['meaning']:
                word_groups[word]['definitions'].append({
                    'pos': row['pos_name'],
                    'pos_abbr': row['abbreviation'],
                    'meaning': row['meaning'],
                    'example': row['example']
                })
    else:
        # 우리말샘 API 결과 처리
        word_groups = {}
        for item in api_results:
            word = item['word']
            if word not in word_groups:
                word_groups[word] = {
                    'word': word,
                    'pronunciation': '',
                    'origin': '',
                    'definitions': []
                }
            
            # sense 정보가 있는 경우
            if 'sense' in item:
                sense = item['sense']
                word_groups[word]['definitions'].append({
                    'pos': sense.get('pos', ''),
                    'pos_abbr': sense.get('pos', ''),
                    'meaning': sense.get('definition', ''),
                    'example': ''
                })
    
    # HTML 생성
    html = f'''
    <!DOCTYPE html>
    <html>
    <head>
        <title>"{query}" 검색결과 - 한국어 사전</title>
        <meta charset="UTF-8">
        <style>
            body {{ font-family: 'Noto Sans KR', Arial, sans-serif; max-width: 900px; margin: 0 auto; padding: 20px; line-height: 1.6; }}
            .header {{ text-align: center; margin-bottom: 30px; border-bottom: 3px solid #4CAF50; padding-bottom: 20px; }}
            .logo {{ color: #4CAF50; font-size: 32px; font-weight: bold; margin-bottom: 15px; }}
            .search-form {{ margin: 20px 0; }}
            .search-input {{ padding: 12px; font-size: 16px; width: 300px; border: 2px solid #ddd; border-radius: 5px; }}
            .search-btn {{ padding: 12px 20px; font-size: 16px; background: #4CAF50; color: white; border: none; border-radius: 5px; cursor: pointer; margin-left: 10px; }}
            .search-btn:hover {{ background: #45a049; }}
            .results-info {{ color: #666; margin: 20px 0; font-size: 18px; }}
            .word-entry {{ border: 1px solid #e0e0e0; margin: 20px 0; padding: 25px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }}
            .word-title {{ font-size: 28px; font-weight: bold; color: #333; margin-bottom: 10px; }}
            .pronunciation {{ color: #666; margin-left: 15px; font-size: 20px; }}
            .origin {{ color: #999; font-size: 14px; margin-bottom: 15px; }}
            .definition {{ margin: 15px 0; padding: 15px; background: #f8f9fa; border-left: 4px solid #4CAF50; border-radius: 4px; }}
            .pos {{ color: #4CAF50; font-weight: bold; margin-right: 10px; }}
            .meaning {{ margin: 8px 0; font-size: 16px; }}
            .example {{ color: #555; font-style: italic; margin-top: 8px; padding: 8px; background: #fff; border-radius: 4px; }}
            .api-source {{ text-align: center; margin-top: 30px; padding: 15px; background: #e8f5e8; border-radius: 5px; color: #2e7d32; }}
        </style>
    </head>
    <body>
        <div class="header">
            <div class="logo">한국어 사전</div>
            <div class="search-form">
                <form action="/search/m/" method="get">
                    <input type="text" name="q" class="search-input" value="{query}" placeholder="검색할 단어를 입력하세요" required>
                    <button type="submit" class="search-btn">검색</button>
                </form>
            </div>
        </div>
        
        <div class="results-info">"{query}" 검색결과 ({len(word_groups)}개)</div>
    '''
    
    for word_data in word_groups.values():
        html += f'''
        <div class="word-entry">
            <div class="word-title">
                {word_data['word']}
                <span class="pronunciation">{word_data['pronunciation'] or ''}</span>
            </div>
            <div class="origin">{word_data['origin'] or ''}</div>
        '''
        
        for i, definition in enumerate(word_data['definitions'], 1):
            html += f'''
            <div class="definition">
                <span class="pos">{definition['pos_abbr'] or definition['pos'] or '명사'}</span>
                <div class="meaning">{i}. {definition['meaning']}</div>
                {f'<div class="example">예: {definition["example"]}</div>' if definition['example'] else ''}
            </div>
            '''
        
        html += '</div>'
    
    # API 사용 여부 표시
    source_info = "우리말샘 API" if api_results else "로컬 데이터베이스"
    html += f'''
        <div class="api-source">
            검색 결과: {source_info} 제공
        </div>
    </body>
    </html>
    '''
    
    return html

# 단어 추가 API
@app.route('/api/add_word', methods=['POST'])
def add_word():
    data = request.json
    word = data.get('word')
    pronunciation = data.get('pronunciation', '')
    origin = data.get('origin', '')
    meaning = data.get('meaning')
    pos_name = data.get('part_of_speech', '명사')
    
    conn = get_db_connection()
    
    try:
        # 단어 삽입
        cursor = conn.execute('''
        INSERT OR IGNORE INTO words (word, pronunciation, origin) 
        VALUES (?, ?, ?)
        ''', (word, pronunciation, origin))
        
        # 단어 ID 가져오기
        word_id = conn.execute('SELECT id FROM words WHERE word = ?', (word,)).fetchone()['id']
        
        # 품사 ID 가져오기
        pos_id = conn.execute('SELECT id FROM parts_of_speech WHERE name = ?', (pos_name,)).fetchone()['id']
        
        # 정의 삽입
        conn.execute('''
        INSERT INTO definitions (word_id, part_of_speech_id, meaning) 
        VALUES (?, ?, ?)
        ''', (word_id, pos_id, meaning))
        
        conn.commit()
        return jsonify({'status': 'success', 'message': '단어가 추가되었습니다.'})
    
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)})
    
    finally:
        conn.close()


In [8]:
# API 키 없이 테스트해보기 (오류 확인)
test_results = search_opendict("가을", 5)
print("API 테스트 결과:", test_results)

# 로컬 데이터베이스 확인
def show_database_contents():
    conn = get_db_connection()
    words = conn.execute('SELECT * FROM words').fetchall()
    print("=== 로컬 단어 목록 ===")
    for word in words:
        print(f"단어: {word['word']}, 발음: {word['pronunciation']}")
    conn.close()

show_database_contents()

API 호출 오류: Expecting value: line 1 column 1 (char 0)
API 테스트 결과: []
=== 로컬 단어 목록 ===
단어: 가을, 발음: [가을]
단어: 봄, 발음: [봄]
단어: 여름, 발음: [여름]
단어: 겨울, 발음: [겨울]
단어: 사랑, 발음: [사랑]
단어: 희망, 발음: [히망]
단어: 학교, 발음: [학꾜]
단어: 컴퓨터, 발음: [컴퓨터]


In [9]:

def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)

# 백그라운드에서 실행
import threading
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

print("우리말샘 API 연동 서버가 시작되었습니다!")
print("브라우저에서 http://localhost:5000 으로 접속하세요")
print("'가을'을 검색해보세요!")

우리말샘 API 연동 서버가 시작되었습니다!
브라우저에서 http://localhost:5000 으로 접속하세요
'가을'을 검색해보세요!
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


In [10]:
@app.route('/api/add_word', methods=['POST'])
def add_word():
    data = request.json
    word = data.get('word')
    pronunciation = data.get('pronunciation', '')
    origin = data.get('origin', '')
    meaning = data.get('meaning')
    pos_name = data.get('part_of_speech', '명사')
    
    conn = get_db_connection()
    
    try:
        # 단어 삽입
        cursor = conn.execute('''
        INSERT OR IGNORE INTO words (word, pronunciation, origin) 
        VALUES (?, ?, ?)
        ''', (word, pronunciation, origin))
        
        # 단어 ID 가져오기
        word_id = conn.execute('SELECT id FROM words WHERE word = ?', (word,)).fetchone()['id']
        
        # 품사 ID 가져오기
        pos_id = conn.execute('SELECT id FROM parts_of_speech WHERE name = ?', (pos_name,)).fetchone()['id']
        
        # 정의 삽입
        conn.execute('''
        INSERT INTO definitions (word_id, part_of_speech_id, meaning) 
        VALUES (?, ?, ?)
        ''', (word_id, pos_id, meaning))
        
        conn.commit()
        return jsonify({'status': 'success', 'message': '단어가 추가되었습니다.'})
    
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)})
    
    finally:
        conn.close()


AssertionError: View function mapping is overwriting an existing endpoint function: add_word

In [11]:
def show_database_contents():
    conn = get_db_connection()
    
    # 전체 단어 목록
    words = conn.execute('SELECT * FROM words').fetchall()
    print("=== 단어 목록 ===")
    for word in words:
        print(f"ID: {word['id']}, 단어: {word['word']}, 발음: {word['pronunciation']}")
    
    print("\n=== 정의 목록 ===")
    definitions = conn.execute('''
    SELECT w.word, d.meaning, p.name as pos_name
    FROM definitions d
    JOIN words w ON d.word_id = w.id
    JOIN parts_of_speech p ON d.part_of_speech_id = p.id
    ''').fetchall()
    
    for def_row in definitions:
        print(f"단어: {def_row['word']}, 품사: {def_row['pos_name']}, 의미: {def_row['meaning']}")
    
    conn.close()

show_database_contents()

=== 단어 목록 ===
ID: 1, 단어: 가을, 발음: [가을]
ID: 2, 단어: 봄, 발음: [봄]
ID: 3, 단어: 여름, 발음: [여름]
ID: 4, 단어: 겨울, 발음: [겨울]
ID: 5, 단어: 사랑, 발음: [사랑]
ID: 6, 단어: 희망, 발음: [히망]
ID: 7, 단어: 학교, 발음: [학꾜]
ID: 8, 단어: 컴퓨터, 발음: [컴퓨터]

=== 정의 목록 ===
단어: 가을, 품사: 명사, 의미: 여름과 겨울 사이의 계절. 음력으로는 7, 8, 9월이고, 양력으로는 9, 10, 11월이다.
단어: 가을, 품사: 명사, 의미: 사물이 무르익은 때를 비유적으로 이르는 말.
단어: 가을, 품사: 명사, 의미: 여름과 겨울 사이의 계절. 음력으로는 7, 8, 9월이고, 양력으로는 9, 10, 11월이다.
단어: 가을, 품사: 명사, 의미: 사물이 무르익은 때를 비유적으로 이르는 말.
단어: 가을, 품사: 명사, 의미: 여름과 겨울 사이의 계절. 음력으로는 7, 8, 9월이고, 양력으로는 9, 10, 11월이다.
단어: 가을, 품사: 명사, 의미: 사물이 무르익은 때를 비유적으로 이르는 말.


127.0.0.1 - - [01/Aug/2025 15:06:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2025 15:07:01] "GET /search/m/?q=사랑 HTTP/1.1" 500 -
Traceback (most recent call last):
  File "/Users/a/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2464, in __call__
    return self.wsgi_app(environ, start_response)
  File "/Users/a/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2450, in wsgi_app
    response = self.handle_exception(e)
  File "/Users/a/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1867, in handle_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/a/opt/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/a/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/a/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
 